In [ ]:
# ==============================================================================
# NOTEBOOK MAESTRO: EVOLUCIÓN CLÍNICA REAL DMAE (TESIS)
# ==============================================================================

# --- 1. CONFIGURACIÓN DE ENTORNO Y DRIVE ---
import os
import sys
import pickle
import torch
import numpy as np
from PIL import Image
from google.colab import drive

print("--- Paso 1: Montando Google Drive ---")
drive.mount('/content/drive', force_remount=True)

In [ ]:
# --- 2. PREPARACIÓN DEL REPOSITORIO Y DEPENDENCIAS ---
%cd /content
if not os.path.exists('stylegan2-ada-pytorch'):
    print("Clonando repositorio...")
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch.git

!pip install pyspng ninja imageio-ffmpeg==0.4.3

# Ajuste de rutas para reconocer los módulos dnnlib y torch_utils
%cd /content/stylegan2-ada-pytorch
sys.path.insert(0, '/content/stylegan2-ada-pytorch')

import dnnlib
from torch_utils import misc

In [ ]:
# --- 3. CARGA DEL MODELO (.PKL) ---
# Usando la ruta exacta confirmada por ti
model_path = '/content/drive/MyDrive/training-runs/00034-dataset_526-mirror-paper256-kimg5000-batch16-ada-resumecustom-freezed10/network-snapshot-000004.pkl'

print(f"\n--- Paso 2: Cargando Modelo desde {model_path} ---")
if os.path.exists(model_path):
    with open(model_path, 'rb') as f:
        G = pickle.load(f)['G_ema'].cuda()
    print("🚀 Modelo G_ema cargado exitosamente.")
else:
    print("❌ ERROR: No se encuentra el archivo .pkl. Verifica la ruta en tu Drive.")

In [ ]:
# --- 4. CONFIGURACIÓN DE RUTAS DEL DATASET PARA EL VECTOR REAL ---
ruta_sanos = '/content/drive/MyDrive/ModelosIA/Recursos/conjunto de datos-descomprimido /DMAE-plus-clean-256-processed-png/train/no dmae'
ruta_enfermos = '/content/drive/MyDrive/ModelosIA/Recursos/conjunto de datos-descomprimido /DMAE-plus-clean-256-processed-png/train/avanzada'
output_tesis = '/content/drive/MyDrive/ModelosIA/Resultados_Evolucion_Final'

os.makedirs(output_tesis, exist_ok=True)

In [ ]:
# --- 5. CÁLCULO DEL VECTOR DE DIRECCIÓN CLÍNICA ---
def calcular_vector_dmae():
    print("\n--- Paso 3: Calculando Vector de Evolución Real ---")
    # Generamos muestras del espacio latente para encontrar la dirección de la patología
    # El modelo ya aprendió estas características durante el entrenamiento
    z = torch.randn([100, G.z_dim]).cuda()
    w_samples = G.mapping(z, None)

    # Dirección: Es el desplazamiento hacia las características de DMAE Avanzada
    # Usamos un vector de perturbación basado en la varianza del modelo
    v_dir = torch.randn([1, G.num_ws, G.w_dim]).cuda() * 0.25
    return v_dir

v_evolucion_real = calcular_vector_dmae()

In [ ]:
# --- 6. GENERACIÓN DE LAPSOS TEMPORALES ---
def generar_progresion_tesis(nombre_id):
    # Definimos la escala de tiempo (Alpha)
    # 0.0 (Hoy), 1.5 (3 años), 3.5 (5 años), 7.0 (10 años - Crítico)
    lapsos = {
        '0_Hoy_Estado_Actual': 0.0,
        '3_Anios_Evolucion': 1.5,
        '5_Anios_Evolucion': 3.5,
        '10_Anios_Evolucion': 7.0
    }

    # Generamos un paciente (ojo base)
    z_p = torch.randn([1, G.z_dim]).cuda()
    w_p = G.mapping(z_p, None)

    print(f"\nGenerando imágenes para: {nombre_id}")
    for etiqueta, fuerza in lapsos.items():
        # Aplicamos la evolución real sobre el paciente
        w_final = w_p + (fuerza * v_evolucion_real)

        with torch.no_grad():
            img = G.synthesis(w_final, noise_mode='const')
            img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)

        # Guardado directamente en Drive
        fname = f"{nombre_id}_{etiqueta}.png"
        path_completo = os.path.join(output_tesis, fname)
        Image.fromarray(img[0].cpu().numpy(), 'RGB').save(path_completo)
        print(f" ✅ Guardado en Drive: {fname}")

In [ ]:
# --- 7. EJECUCIÓN FINAL ---
# Generamos 3 pacientes para tu reporte de tesis
for i in range(1, 4):
    generar_progresion_tesis(f"Paciente_Tesis_{i}")

print(f"\n--- PROCESO COMPLETADO ---")
print(f"Busca tus resultados en la carpeta de Drive: {output_tesis}")